# Step 2: Multimodal Page Classification with Boundary Detection

This notebook demonstrates how to classify document pages and detect page-level document boundaries using the `multimodalPageLevelClassification` method.

**Key Features:**
- Multimodal classification using both text and images
- Document boundary detection (start/continue)
- Section creation based on both document type changes AND boundary markers

**Inputs:**
- Document object with OCR results from Step 1
- Configuration with boundary detection enabled

**Outputs:**
- Document with classifications and boundary markers
- Sections respecting document boundaries

## 1. Load Libraries and Configure Logging

In [1]:
import os
import json
import yaml
import time
import logging
from pathlib import Path

# Import IDP libraries
from idp_common.classification.service import ClassificationService
from idp_common.models import Document, Status

# Configure logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger('idp_common.classification').setLevel(logging.INFO)
logging.getLogger('idp_common.bedrock.client').setLevel(logging.INFO)

print("Libraries imported successfully")

Libraries imported successfully


## 2. Load OCR Output and Configuration

In [2]:
# Load OCR output from Step 1
examples_dir = Path.cwd()
ocr_data_path = examples_dir / 'data' / 'ocr_output.json'

# Check if OCR output exists
if not ocr_data_path.exists():
    # Try alternative path from step1_ocr
    ocr_data_path = examples_dir / '.data' / 'step1_ocr' / 'document.json'
    
if not ocr_data_path.exists():
    raise FileNotFoundError(f"OCR output not found at {ocr_data_path}")

with open(ocr_data_path) as f:
    doc_data = json.load(f)
    
# Convert to Document object
if isinstance(doc_data, str):
    document = Document.from_json(doc_data)
else:
    document = Document.from_dict(doc_data) if 'id' in doc_data else Document.from_json(json.dumps(doc_data))

print(f"Loaded document: {document.id}")
print(f"Number of pages: {document.num_pages}")

# Load configuration directly from config files
config_dir = Path("config")
CONFIG = {}

# Load each configuration file
config_files = [
    "classification.yaml", 
    "classes.yaml"
]

for config_file in config_files:
    config_path = config_dir / config_file
    if config_path.exists():
        with open(config_path, 'r') as f:
            file_config = yaml.safe_load(f)
            CONFIG.update(file_config)
        print(f"Loaded {config_file}")
    else:
        print(f"Warning: {config_file} not found")

# Use CONFIG as the main config object
config = CONFIG

# Load environment info from previous step
env_path = ocr_data_path.parent / "environment.json"
with open(env_path, 'r') as f:
    env_info = json.load(f)

# Display configuration details
classification_config = config.get('classification', {})
print("\nClassification Configuration:")
print(f"Method: {classification_config.get('classificationMethod')}")
print(f"Model: {classification_config.get('model', config.get('model_id'))}")
print(f"Temperature: {classification_config.get('temperature')}")

# Display available document classes
classes = config.get('classes', [])
print(f"\nAvailable Document Classes: {len(classes)}")
for cls in classes:
    print(f"- {cls['name']}: {cls['description'][:100]}...")

Loaded document: bank_statement
Number of pages: 6
Loaded classification.yaml
Loaded classes.yaml

Classification Configuration:
Method: multimodalPageLevelClassification
Model: us.amazon.nova-pro-v1:0
Temperature: 0.0

Available Document Classes: 6
- Payslip: An employee wage statement showing earnings, deductions, taxes, and net pay for a specific pay perio...
- US-drivers-licenses: An official government-issued identification document that authorizes an individual to operate  moto...
- Bank-checks: A written financial instrument directing a bank to pay a specific amount of money from  the account ...
- Bank-Statement: A periodic financial document issued by banks detailing account activity, balances,  and transaction...
- W2: An annual tax document provided by employers to employees reporting wages earned and  taxes withheld...
- Homeowners-Insurance-Application: An application form for homeowners insurance coverage containing applicant personal information,  pr...


## 3. Initialize Classification Service

In [3]:
# Set AWS region if not already set
if 'AWS_REGION' not in os.environ:
    os.environ['AWS_REGION'] = 'us-west-2'

# Create classification service
service = ClassificationService(
    region=os.environ.get('AWS_REGION', 'us-west-2'), 
    config=config,
    backend='bedrock'  # Using Bedrock for multimodal classification
)

print("Classification service initialized")
print(f"Using method: {service.classification_method}")

INFO:idp_common.classification.service:Classification caching disabled
INFO:idp_common.classification.service:Initialized classification service with Bedrock backend using model us.amazon.nova-pro-v1:0
INFO:idp_common.classification.service:Using multimodal page-level classification method with document boundary detection


Classification service initialized
Using method: multimodalPageLevelClassification


## 4. Classify Document with Boundary Detection

In [4]:
# Classify the document
print("\nClassifying document with boundary detection...")
start_time = time.time()

try:
    classified_document = service.classify_document(document)
    classification_time = time.time() - start_time
    
    print(f"✅ Classification completed in {classification_time:.2f} seconds")
    print(f"Document status: {classified_document.status.value}")
    
except Exception as e:
    print(f"❌ Classification failed: {str(e)}")
    raise

INFO:idp_common.classification.service:Classifying document with 6 pages using multimodal page-level classification with bedrock backend
INFO:idp_common.classification.service:Attempting to retrieve cached page classifications for document bank_statement
INFO:idp_common.classification.service:Found 0 cached page classifications, classifying 6 remaining pages



Classifying document with boundary detection...


INFO:idp_common.classification.service:Classifying page 1 with Bedrock
INFO:idp_common.classification.service:Classifying page 5 with Bedrock
INFO:idp_common.classification.service:Classifying page 4 with Bedrock
INFO:idp_common.classification.service:Classifying page 2 with Bedrock
INFO:idp_common.classification.service:Classifying page 3 with Bedrock
INFO:idp_common.classification.service:Classifying page 6 with Bedrock
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.amazon.nova-pro-v1:0
INFO:idp_common.bedrock.client:Bedrock request attempt 1/7:
INFO:idp_common.bedrock.client:  - model: us.amazon.nova-pro-v1:0
INFO:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0.0, 'topP': 0.1, 'maxTokens': 4096}
INFO:idp_common.bedrock.client:  - system: [{'text': 'You are a multimodal document classification expert that analyzes business documents using both visual la

✅ Classification completed in 8.23 seconds
Document status: QUEUED


## 5. Display Results with Boundary Information

In [5]:
# Show sections with boundary information
if classified_document.sections:
    print(f"\n📑 Detected {len(classified_document.sections)} sections:")
    for section in classified_document.sections:
        print(f"\nSection {section.section_id}:")
        print(f"  Type: {section.classification}")
        print(f"  Pages: {section.page_ids}")
        print(f"  Number of pages: {len(section.page_ids)}")
else:
    print("\n⚠️ No sections detected")

# Show page-level classifications with boundary markers
print("\n📄 Page-level classifications with boundaries:")
for page_id in sorted(classified_document.pages.keys(), key=lambda x: int(x) if x.isdigit() else float('inf')):
    page = classified_document.pages[page_id]
    
    # Get boundary information from page metadata if available
    boundary = "unknown"
    if hasattr(page, 'metadata') and page.metadata:
        boundary = page.metadata.get('document_boundary', 'unknown')
    
    # Check if page has classification result in Document model
    if hasattr(page, 'classification'):
        print(f"Page {page_id}: {page.classification} [boundary: {boundary}]")


📑 Detected 6 sections:

Section 1:
  Type: Payslip
  Pages: ['1']
  Number of pages: 1

Section 2:
  Type: US-drivers-licenses
  Pages: ['2']
  Number of pages: 1

Section 3:
  Type: Bank-checks
  Pages: ['3']
  Number of pages: 1

Section 4:
  Type: Bank-Statement
  Pages: ['4']
  Number of pages: 1

Section 5:
  Type: W2
  Pages: ['5']
  Number of pages: 1

Section 6:
  Type: Homeowners-Insurance-Application
  Pages: ['6']
  Number of pages: 1

📄 Page-level classifications with boundaries:
Page 1: Payslip [boundary: start]
Page 2: US-drivers-licenses [boundary: start]
Page 3: Bank-checks [boundary: start]
Page 4: Bank-Statement [boundary: start]
Page 5: W2 [boundary: start]
Page 6: Homeowners-Insurance-Application [boundary: start]


## 6. Analyze Boundary Detection Results

In [6]:
# Analyze boundary detection effectiveness
print("\n🔍 Boundary Detection Analysis:")

# Count boundary markers
boundary_starts = 0
boundary_info = {}

for page_id, page in classified_document.pages.items():
    # Try to get boundary info from metadata
    boundary = "unknown"
    if hasattr(page, 'metadata') and page.metadata:
        boundary = page.metadata.get('document_boundary', 'unknown')
    
    boundary_info[page_id] = boundary
    if boundary == 'start':
        boundary_starts += 1

print(f"Number of 'start' boundaries detected: {boundary_starts}")
print(f"Number of sections created: {len(classified_document.sections) if classified_document.sections else 0}")

# Display boundary transitions
print("\n📊 Boundary Transitions:")
prev_type = None
for page_id in sorted(classified_document.pages.keys(), key=lambda x: int(x) if x.isdigit() else float('inf')):
    page = classified_document.pages[page_id]
    if hasattr(page, 'classification'):
        curr_type = page.classification
        boundary = boundary_info.get(page_id, 'unknown')
        
        if prev_type and prev_type != curr_type:
            print(f"  Type change at page {page_id}: {prev_type} → {curr_type} [boundary: {boundary}]")
        elif boundary == 'start':
            print(f"  Boundary marker at page {page_id}: {curr_type} [boundary: start]")
        
        prev_type = curr_type

# Verify boundary logic
if classified_document.sections and boundary_starts > 0:
    print("\n✅ Boundary detection is working - sections align with boundary markers")
else:
    print("\n⚠️ No boundary markers detected - sections based on document type changes only")


🔍 Boundary Detection Analysis:
Number of 'start' boundaries detected: 6
Number of sections created: 6

📊 Boundary Transitions:
  Boundary marker at page 1: Payslip [boundary: start]
  Type change at page 2: Payslip → US-drivers-licenses [boundary: start]
  Type change at page 3: US-drivers-licenses → Bank-checks [boundary: start]
  Type change at page 4: Bank-checks → Bank-Statement [boundary: start]
  Type change at page 5: Bank-Statement → W2 [boundary: start]
  Type change at page 6: W2 → Homeowners-Insurance-Application [boundary: start]

✅ Boundary detection is working - sections align with boundary markers


## 7. Save Results

In [10]:
# Create output directory
output_dir = examples_dir / 'data' / 'step2_classification'
output_dir.mkdir(parents=True, exist_ok=True)

# Save classified document
output_path = output_dir / 'document.json'
with open(output_path, 'w') as f:
    f.write(classified_document.to_json())
    
print(f"\n💾 Saved classified document to: {output_path}")

# Save classification summary
summary = {
    'document_id': classified_document.id,
    'classification_method': service.classification_method,
    'processing_time_seconds': classification_time,
    'num_sections': len(classified_document.sections) if classified_document.sections else 0,
    'sections': [
        {
            'section_id': section.section_id,
            'classification': section.classification,
            'page_ids': section.page_ids,
            'num_pages': len(section.page_ids)
        } for section in (classified_document.sections or [])
    ],
    'boundary_detection': {
        'enabled': True,
        'num_start_boundaries': boundary_starts,
        'boundary_info': boundary_info
    },
    'page_classifications': {
        page_id: {
            'classification': page.classification if hasattr(page, 'classification') else 'unknown',
            'boundary': boundary_info.get(page_id, 'unknown')
        } for page_id, page in classified_document.pages.items()
    }
}

summary_path = output_dir / 'classification_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
    
print(f"💾 Saved classification summary to: {summary_path}")

# Save configuration (pass through)
config_path = output_dir / "config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

# Save environment info (pass through)
env_path = output_dir / "environment.json"
with open(env_path, 'w') as f:
    json.dump(env_info, f, indent=2)

print(f"💾 Saved configuration to: {config_path}")
print(f"💾 Saved environment info to: {env_path}")


💾 Saved classified document to: /home/ec2-user/projects/genaiic-idp-accelerator-2/notebooks/examples/.data/step2_classification/classified_document.json
💾 Saved classification summary to: /home/ec2-user/projects/genaiic-idp-accelerator-2/notebooks/examples/.data/step2_classification/classification_summary.json


## 8. Summary

In [8]:
print("\n" + "="*60)
print("✅ BOUNDARY CLASSIFICATION COMPLETE")
print("="*60)
print(f"Document ID: {classified_document.id}")
print(f"Classification Method: {service.classification_method}")
print(f"Processing Time: {classification_time:.2f} seconds")
print(f"Sections Identified: {len(classified_document.sections) if classified_document.sections else 0}")
print(f"Boundary Markers Found: {boundary_starts}")
print(f"\n📁 Output saved to: {output_dir}")
print("\n📌 Next step: Run extraction on the classified sections")


✅ BOUNDARY CLASSIFICATION COMPLETE
Document ID: bank_statement
Classification Method: multimodalPageLevelClassification
Processing Time: 8.23 seconds
Sections Identified: 6
Boundary Markers Found: 6

📁 Output saved to: /home/ec2-user/projects/genaiic-idp-accelerator-2/notebooks/examples/data/classification_boundary_output

📌 Next step: Run extraction on the classified sections
